In [7]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '../')

# import embedding methods
from embedding.bernoulli import Bernoulli
from embedding.KL import KL

# import evaluation methods
from evaluation import evaluate_link_prediction

# import utils
from utils import graph_util
from utils import plot_util

# visualization
%matplotlib inline
#sns.set_style('whitegrid')

import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Experiment 1

In [15]:
# Experiment 1

# create folder to store results of experiment
result_folder = plot_util.create_experiment_folder()
print(f'The results of the current experiment are stored at experiments/{result_folder}')

exp = {
    "emb_dim": 64,
    "max_epoch": 500,
    "learning_rate": 1e-2, #Adam
    "weight_decay": 1e-7,
    "number_rounds_link_prediction": 3
}


# pick datasets
datasets = ["conra", "hvr"]

# initialize embedding methods
b = Bernoulli(embedding_dimension=exp["emb_dim"], distance_meassure='sigmoid', max_epoch=exp["max_epoch"])
kl = KL(embedding_dimension=exp["emb_dim"], distance_meassure='sigmoid',max_epoch=exp["max_epoch"])

embedding_methods = [b, kl]

# save experiment readme summarizing the experiment setup

for dataset in datasets:
    
    # load dataset
    A, y = graph_util.load_dataset(dataset)
    
    for embedding_method in embedding_methods:
        
        # do link prediction
        link_prediction_folder = result_folder + "/link_prediction"
        evaluate_link_prediction.expLP(A,dataset,embedding_method,exp["number_rounds_link_prediction"],link_prediction_folder, train_ratio=0.8,undirected=True)
        
        # do node classification
        
        # do graph reconstruction
        
        # do down projection and visualization

The results of the current experiment are stored at experiments/results/2019_11_21_23_05
	Link Prediction is started...
	Link Prediction is started...


RuntimeError: expected device cuda:0 but got device cpu